In [1]:
import torch
import numpy as np
import os
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader

In [2]:
device="mps" if torch.backends.mps.is_available() else "cpu"
device

'mps'

In [3]:
n="40"
DATA_PATH=f"data/Embedings/Embeding-{n}"
TRAIN_EMBEDS = os.path.join(DATA_PATH, f'trainEmbeds{n}.npz')
TEST_EMBEDS = os.path.join(DATA_PATH, f'testEmbeds{n}.npz')
trainEmbeds, trainLabels = np.load(TRAIN_EMBEDS, allow_pickle=True).values()
testEmbeds, testLabels = np.load(TEST_EMBEDS, allow_pickle=True).values()
trainEmbeds.shape

(38786, 512)

In [4]:
import pickle
with open('data/class_list.pkl', 'rb') as file:
    ClassList = pickle.load(file)
print(len(ClassList))    
def name_from_index(i, ClassList=ClassList):
    reversed_dict = {v: k for k, v in ClassList.items()}
    if isinstance(i, np.ndarray):  # Check if i is a NumPy array
        names = [reversed_dict.get(idx.item(), None) for idx in i]
        return names
    else:
        return reversed_dict.get(i, None)

20


In [5]:
import torch.nn as nn
import torch.optim as optim
X = torch.tensor(trainEmbeds, dtype=torch.float32).to(device)
y = torch.tensor(trainLabels, dtype=torch.long).to(device)
X_test=torch.tensor(testEmbeds,dtype=torch.float32).to(device)
y_test=torch.tensor(testLabels,dtype=torch.long).to(device)

# Defining SVM model

In [16]:
class KernelSVM(torch.nn.Module):
    def __init__(self, train_data_x, kernel='rbf',
                 gamma_init=1.0, train_gamma=True):
        super().__init__()
        assert kernel in ['linear', 'rbf']
        self._train_data_x = train_data_x
        
        if kernel == 'linear':
            self._kernel = self.linear
            self._num_c = 2
        elif kernel == 'rbf':
            self._kernel = self.rbf
            self._num_c = len(ClassList)
            self._gamma = torch.nn.Parameter(torch.FloatTensor([gamma_init]),
                                             requires_grad=train_gamma)
        else:
            assert False
            
        self._w = torch.nn.Linear(in_features=self._num_c, out_features=1)

    def rbf(self, x, gamma=1):
        y = self._train_data_x.repeat(x.size(0), 1, 1)
        return torch.exp(-self._gamma*((x[:,None]-y)**2).sum(dim=2))
    
    @staticmethod
    def linear(x):
        return x

    def forward(self, x):
        y = self._kernel(x)
        y = self._w(y)
        return y
model = KernelSVM(X, kernel='rbf')

# Loss function and Optimizer function

In [17]:
loss_func = nn.CrossEntropyLoss()
# nn.NLLLoss()
admoptimizer = optim.Adam(model.parameters(), lr=0.051)

# Accuracy Function

In [18]:
def acc(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item() # torch.eq() calculates where two tensors are equal
    acc = (correct / len(y_pred)) * 100 
    return acc

# Training Loop

In [19]:
def modelSave(model):
    from pathlib import Path

    # 1. Create models directory 
    MODEL_PATH = Path(f"data/classLen- {len(ClassList)}/embeding-{n}")
    MODEL_PATH.mkdir(parents=True, exist_ok=True)

    # 2. Create model save path 
    MODEL_NAME = f"predictFacesModelAcc{testAcc:.2f}_With{n}Embeds.pth"
    MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

    # 3. Save the model state dict 
    print(f"Saving model to: {MODEL_SAVE_PATH}")
    torch.save(obj=model.state_dict(), # only saving the state_dict() only saves the models learned parameters
               f=MODEL_SAVE_PATH)

In [20]:
%%time
num_epochs = 2400000
trainingLoss= []
testingLoss =1
trainAcc = []
testAcc =0
for epoch in range(num_epochs):
    outputs = model(X)
    _, y_predict = torch.max(outputs, 1)
    train_accuracy = acc(y, y_predict)
    train_loss = loss_func(outputs, y)

    admoptimizer.zero_grad()
    train_loss.backward()
    admoptimizer.step()
    model.eval()
    with torch.no_grad():
        predictions = model(X_test)
        _, pred_labels = torch.max(predictions, 1)
        y_test = y_test.to(torch.long)  # Convert to Long data type
        test_accuracy = acc(y_test, pred_labels)
        test_loss = loss_func(predictions, y_test)
    if epoch % 10 == 0:
        if testAcc<test_accuracy:
            testAcc=test_accuracy
            if testAcc>90:
                modelSave(model)
        if test_loss<testingLoss and testAcc>90:
            testingLoss=test_loss-0.001
            modelSave(model)
            
        print(f"Epoch: {epoch} | TrainingLoss= {train_loss} | TestingLoss= {test_loss} | Accuracy= {test_accuracy}")


RuntimeError: Invalid buffer size: 2869.33 GB

In [ ]:
from pathlib import Path

    # 1. Create models directory 
    MODEL_PATH = Path(f"data/classLen- {len(ClassList)}/embeding-{n}")
    MODEL_PATH.mkdir(parents=True, exist_ok=True)

    # 2. Create model save path 
    MODEL_NAME = f"predictFacesModelAcc{testAcc:.2f}_With{n}Embeds.pth"
    MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

    # 3. Save the model state dict 
    print(f"Saving model to: {MODEL_SAVE_PATH}")
    torch.save(obj=model.state_dict(), # only saving the state_dict() only saves the models learned parameters
               f=MODEL_SAVE_PATH)